In [ ]:
from skimage.io import imread_collection
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer,accuracy_score
from scipy.stats import randint as sp_randint
from progressbar import progressbar
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
Glaucoma = imread_collection("../../../Documents/Bases/RIMONE_v2/Training/Glaucoma/*.jpg")
Normal = imread_collection("../../../Documents/Bases/RIMONE_v2/Training/Normal/*.jpg")

In [ ]:
def normalize(valor):
    return int(valor)/255

def getMinimum(image):
    return image.min()

In [ ]:
def phylogeny(img):

    prodind = 0.0
    dis = 0
    valorMPD = 0.0
    SPD = 0.0
    S1 = 0.0
    S2 = 0.0
    PD = 0.0
    soma1 = 0
    somatorio = 0
    MNND = 0.0
    q = 0.0

    especie_ = 0
    matFeat = []

    img = rgb2gray(img)*255
    img = img.astype(int)

    min = getMinimum(image=img)

    sizeHistograma = 256
    histograma = [0]*256
    distance = [0]*256

    # -----------------------------Filogenéticos-------------------------------------
    background = min

    especie = 0

    for b in range(img.shape[0]):
        for c in range(img.shape[1]):
            value = img[b][c]
            if int(value) > 0:
                histograma[value] += 1

    for d in range(0, sizeHistograma):
        if int(histograma[d]) > 0:
            especie_ += int(histograma[d])

    for d in range(0, sizeHistograma):
        if int(histograma[d]) > 0:
            especie += 1


    for x in range(0, sizeHistograma):
        for y in range(x + 1, sizeHistograma):
            if x == 0:
                distance[x] = (y - x + 1)
            else:
                distance[x] = (y - x + 2)

            prodind = (distance[x] * histograma[x] * histograma[y])
            somatorio += (histograma[x] * histograma[y])

            for q in range(x, y):
                MNND = MNND + (distance[x] * histograma[q])

    t = ((sizeHistograma - 1) / 2.0)
    total = sizeHistograma * t

    for x in range(0, sizeHistograma):
        for y in range(x + 1, sizeHistograma):
            if x == 0:
                dis = (y - x) + 1
            else:
                dis = (y - x) + 2

            prodind += (dis * histograma[x] * histograma[y])
            somatorio += (histograma[x] * histograma[y])

            for indice in range(x, (y + 1)):
                soma1 += histograma[indice]

            Ai = 0.0
            Ai = soma1 / ((y - x) + 1)
            S1 += (dis * Ai)
            S2 += Ai

        valorMPD += prodind

    PD = S1 / S2
    MPD = valorMPD / somatorio
    SPD = total * MPD

    # -----------------------------Taxonômicos--------------------------------------
    somatorioDelta = 0.0
    for x in range(0, sizeHistograma):
        somatorio_initial = 0.0
        for y in range(x + 1, sizeHistograma):
            val = histograma[x] * histograma[y] * ((y - x) + 1)
            somatorio_initial += val
        somatorioDelta += somatorio_initial
    deLta = somatorioDelta / (especie_ * (especie_ - 1) / 2)

    somatorioDelta_ = 0.0
    somatorioDelta_q = 0.0
    for x in range(0, sizeHistograma):
        somatorio_initial_ = 0.0
        somatorio_initialq = 0.0
        for y in range(x + 1, sizeHistograma):
            val = histograma[x] * histograma[y] * ((y - x) + 1)
            valq = histograma[x] * histograma[y]
            somatorio_initial_ += val
            somatorio_initialq += valq
        somatorioDelta_ += somatorio_initial_
        somatorioDelta_q += somatorio_initialq
    delta_ = somatorioDelta_ / somatorioDelta_q
    
    props = np.zeros((6))
    props[0] = delta_
    props[1] = deLta
    props[2] = MNND
    props[3] = PD
    props[4] = SPD
    props[5] = MPD
    
    return props

In [ ]:
def normalize(x):#
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [ ]:
labels = np.concatenate((np.ones(len(Glaucoma)),np.zeros(len(Normal))))
features = np.zeros((len(labels),6))

for id_img, img in enumerate(progressbar(Glaucoma)):
    features[id_img] = phylogeny(img)

for id_img, img in enumerate(progressbar(Normal)):
    features[id_img+len(Glaucoma)] = phylogeny(img)

In [ ]:
features = normalize(features)

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(features,labels, test_size= 0.10, random_state=27)

In [ ]:
def rf_parameter_estimation(xEst, yEst):
    
    clf = RandomForestClassifier(n_estimators=20)    
    # specify parameters and distributions to sample from
    hyperparameters = {"n_estimators": range(10,1000,50),
                  "max_depth": range(1,100),
                  "max_features": sp_randint(1, xEst.shape[1]),
                  "min_samples_split": sp_randint(2, xEst.shape[1]),
                  "min_samples_leaf": sp_randint(1, xEst.shape[1]),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    
    
    # run randomized search
    n_iter_search = 20
    random_search = RandomizedSearchCV(clf, param_distributions=hyperparameters,
                                       n_iter=n_iter_search,scoring=make_scorer(accuracy_score))
    
    
    random_search.fit(xEst, yEst)
    report(random_search.cv_results_)
    return random_search.best_params_

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))

In [ ]:
parameters = rf_parameter_estimation(X_train, y_train)
c_rf = RandomForestClassifier(**parameters)
c_rf.fit(X_train,y_train)
pred = c_rf.predict(x_test)
acc = accuracy_score(y_test, pred)

In [ ]:
acc